# Outil de création du fichier _rawdata_

Le fichier _rawdata_ copie une partie des séries importées d'Eurostat et de GED pour les réinjecter dans le template Excel qui servait auparavant à les collecter.

Le but de cette opération est de pouvoir opérer certains calculs qui étaient réalisés directement dans le template, en amont de la mise à jour de la base Collecte.

## Importation des modules nécessaires

* Pandas pour l'importation des données (csv pour Eurostat, Oracle pour GED) et la manipulation des données dans des dataframes ;
* os pour accéder aux ;
* la fonction create_engine du package sqlalchemy pour créer la connexion aux bases d'Oracle.

In [160]:
import pandas as pd
import os
from sqlalchemy import create_engine

## Variables

On définit:
* une année de début et de fin pour les données à récupérer dans Oracle ;
* la liste des pays dont on souhaite récupérer les données ;
* la liste des séries qu'on veut injecter dans RawData.

In [161]:
deb = 1990
fin = 2022

l_pays = ['aut', 'bel', 'bgr', 'cro', 'cyp', 'dnk', 'esp', 'eso', 'fin', 'fra', 'gbr', 'grc', 'hun', 'irl', 'ita', 'lat', \
           'lth', 'lux', 'mlt', 'nld', 'nor', 'pol', 'prt', 'rcz', 'rfa', 'rfy', 'rom', 'rsl', 'slo', 'sui', 'swe', \
           'ueur', 'ueur28']

l_series = ['carcfado', 'chacfagr', 'chacfboi', 'chacfchi', 'chacfcon', 'chacfdvr', 'chacfiaa', 'chacfidv', 'chacfima', 'chacfind', 'chacfmac', 'chacfmin',
            'chacfmnf', 'chacfmnm', 'chacfppp', 'chacfres', 'chacfret', 'chacfsid', 'chacfter', 'chacftex', 'chacfveh', 'cmscfagr', 'cmscfboi', 'cmscfchi',
            'cmscfcon', 'cmscfdvr', 'cmscfiaa', 'cmscfidv', 'cmscfima', 'cmscfind', 'cmscfmac', 'cmscfmin', 'cmscfmnf', 'cmscfmnm', 'cmscfppp', 'cmscfres',
            'cmscfret', 'cmscfsid', 'cmscfter', 'cmscftex', 'cmscftra', 'cmscfveh', 'cmscp', 'cmsie', 'cokcfboi', 'cokcfchi', 'cokcfcon', 'cokcfiaa', 'cokcfidv',
            'cokcfima', 'cokcfind', 'cokcfmac', 'cokcfmin', 'cokcfmnf', 'cokcfmnm', 'cokcfppp', 'cokcfsid', 'cokcftex', 'cokcfveh', 'cpr', 'cprxx', 'dj', 'djcli',
            'djfrecli', 'djref', 'edvpd', 'edvpd1', 'eeopd', 'eeopd1', 'egepd', 'ehypd', 'elecfagr', 'elecfboi', 'elecfchi', 'elecfcon', 'elecfdvr', 'elecffer',
            'elecfiaa', 'elecfidv', 'elecfima', 'elecfind', 'elecfmac', 'elecfmin', 'elecfmnf', 'elecfmnm', 'elecfppp', 'elecfres', 'elecfret', 'elecfrou', 'elecfsid',
            'elecfter', 'elecftex', 'elecftra', 'elecfveh', 'eleimex', 'elepd1', 'elepdhy', 'elepdnu', 'elepdth', 'empagr', 'empbup', 'empbur', 'empcom', 'empdiv',
            'empedu', 'empgov', 'emphcr', 'emphos', 'empter', 'enccfagr', 'enccfboi', 'enccfchi', 'enccfcon', 'enccfdvr', 'enccfiaa', 'enccfidv', 'enccfima', 'enccfind',
            'enccfmac', 'enccfmin', 'enccfmnf', 'enccfmnm', 'enccfppp', 'enccfres', 'enccfres1', 'enccfres2', 'enccfret', 'enccfrou', 'enccfsid', 'enccfter', 'enccftex',
            'enccftra', 'enccfveh', 'enccp', 'encie', 'enupd', 'esacfair', 'esscfflv', 'esscfrou', 'esscftra', 'folcfagr', 'folcfboi', 'folcfchi', 'folcfcon', 'folcfdvr',
            'folcffer', 'folcfflv', 'folcfiaa', 'folcfidv', 'folcfima', 'folcfind', 'folcfmac', 'folcfmin', 'folcfmnf', 'folcfmnm', 'folcfppp', 'folcfres', 'folcfret',
            'folcfsid', 'folcfter', 'folcftex', 'folcftra', 'folcfveh', 'gadcfboi', 'gadcfchi', 'gadcfcon', 'gadcfiaa', 'gadcfidv', 'gadcfima', 'gadcfmac', 'gadcfmin',
            'gadcfmnf', 'gadcfmnm', 'gadcfppp', 'gadcfsid', 'gadcftex', 'gadcfveh', 'gazcfagr', 'gazcfboi', 'gazcfchi', 'gazcfcon', 'gazcfdvr', 'gazcfiaa', 'gazcfidv',
            'gazcfima', 'gazcfind', 'gazcfmac', 'gazcfmin', 'gazcfmnf', 'gazcfmnm', 'gazcfppp', 'gazcfres', 'gazcfret', 'gazcfsid', 'gazcfter', 'gazcftex', 'gazcftra',
            'gazcfveh', 'gazcp', 'gazie', 'glecfrou', 'glgcfrou', 'gnacfboi', 'gnacfchi', 'gnacfcon', 'gnacfiaa', 'gnacfidv', 'gnacfima', 'gnacfmac', 'gnacfmin',
            'gnacfmnf', 'gnacfmnm', 'gnacfppp', 'gnacfrou', 'gnacfsid', 'gnacftex', 'gnacfveh', 'gplcfagr', 'gplcfdvr', 'gplcfres', 'gplcfret', 'gplcfrou', 'gplcfter',
            'gplcftra', 'gzlcfboi', 'gzlcfchi', 'gzlcfcon', 'gzlcffer', 'gzlcfflv', 'gzlcfiaa', 'gzlcfidv', 'gzlcfima', 'gzlcfind', 'gzlcfmac', 'gzlcfmin', 'gzlcfmnf',
            'gzlcfmnm', 'gzlcfppp', 'gzlcfrou', 'gzlcfsid', 'gzlcftex', 'gzlcftra', 'gzlcfveh', 'holcfagr', 'holcfdvr', 'holcfres', 'holcfret', 'holcfte', 'ipiboixx',
            'ipichixx', 'ipicmpxx', 'ipiconxx', 'ipifabxx', 'ipiiaaxx', 'ipiidvxx', 'ipiimaxx', 'ipiindxx', 'ipimacxx', 'ipimanxx', 'ipiminxx', 'ipimnmxx', 'ipimprxx',
            'ipipapxx', 'ipipppxx', 'ipitexxx', 'ipivehxx', 'ligcfagr', 'ligcfboi', 'ligcfchi', 'ligcfcon', 'ligcfdvr', 'ligcfiaa', 'ligcfidv', 'ligcfima', 'ligcfind',
            'ligcfmac', 'ligcfmin', 'ligcfmnf', 'ligcfmnm', 'ligcfppp', 'ligcfres', 'ligcfret', 'ligcfsid', 'ligcfter', 'ligcftex', 'ligcfveh', 'osfcfboi', 'osfcfchi',
            'osfcfcon', 'osfcfiaa', 'osfcfidv', 'osfcfima', 'osfcfind', 'osfcfmac', 'osfcfmin', 'osfcfmnf', 'osfcfmnm', 'osfcfppp', 'osfcfsid', 'osfcftex', 'osfcfveh',
            'paccf', 'pdvcfboi', 'pdvcfchi', 'pdvcfcon', 'pdvcfiaa', 'pdvcfidv', 'pdvcfima', 'pdvcfind', 'pdvcfmac', 'pdvcfmin', 'pdvcfmnf', 'pdvcfmnm', 'pdvcfppp',
            'pdvcfsid', 'pdvcftex', 'pdvcfveh', 'petcfagr', 'petcfboi', 'petcfchi', 'petcfcon', 'petcfdvr', 'petcfiaa', 'petcfidv', 'petcfima', 'petcfind', 'petcfmac',
            'petcfmin', 'petcfmnf', 'petcfmnm', 'petcfppp', 'petcfres', 'petcfret', 'petcfsid', 'petcfter', 'petcftex', 'petcftra', 'petcfveh', 'petcp', 'petie', 'pib',
            'pibxx', 'pop', 'prdacb', 'prdacbele', 'prdacboxy', 'prdamm', 'prdcim', 'prdpap', 'prdpat', 'solpd', 'toccfado', 'toccfagr', 'toccfain', 'toccfboi', 'toccfchi',
            'toccfcon', 'toccfdvr', 'toccffer', 'toccfflv', 'toccfiaa', 'toccfidv', 'toccfima', 'toccfind', 'toccfmac', 'toccfmin', 'toccfmnf', 'toccfmnm', 'toccfnen',
            'toccfppp', 'toccfres', 'toccfret', 'toccfrou', 'toccfsid', 'toccfter', 'toccftex', 'toccftra', 'toccfveh', 'toccp', 'totie', 'toucfagr', 'toucfdvr', 'toucfres',
            'toucfret', 'toucfter', 'txchgeuro', 'txchgppp', 'vadagr', 'vadagrxx', 'vadboi', 'vadboixx', 'vadbupxx', 'vadburxx', 'vadchi', 'vadchixx', 'vadcmp', 'vadcmpxx',
            'vadcomxx', 'vadcon', 'vadconxx', 'vaddivxx', 'vadeduxx', 'vadegw', 'vadegwxx', 'vadfab', 'vadfabxx', 'vadgovxx', 'vadhcrxx', 'vadhosxx', 'vadiaa', 'vadiaaxx',
            'vadidv', 'vadidvxx', 'vadind', 'vadindxx', 'vadmac', 'vadmacxx', 'vadman', 'vadmanxx', 'vadmin', 'vadminxx', 'vadmnm', 'vadmnmxx', 'vadmpr', 'vadmprxx',
            'vadpap', 'vadpapxx', 'vadppp', 'vadpppxx', 'vadref', 'vadrefxx', 'vadter', 'vadterxx', 'vadtex', 'vadtexxx', 'vadveh', 'vadvehxx', 'vapcfagr', 'vapcfboi',
            'vapcfchi', 'vapcfcon', 'vapcfdvr', 'vapcfiaa', 'vapcfidv', 'vapcfima', 'vapcfind', 'vapcfmac', 'vapcfmin', 'vapcfmnf', 'vapcfmnm', 'vapcfppp', 'vapcfres',
            'vapcfret', 'vapcfsid', 'vapcfter', 'vapcftex', 'vapcfveh', 'vappd']

## Fonctions

On définit également deux fonctions servant à calculer les quatre séries qui n'étaient pas directement disponibles dans Eurostat :
* _djref_ et _djrefcli_ calculées comme moyennes par pays de _dj_ et _djcli_ respectivement ;
* _ipipppxx_ et _ipiidvxx_ qui sont des moyennes pondérées.

In [162]:
def moy_simple(df, serieR, serieS, bornes=[deb,fin]):
    dfAjout = df[(df.serie == serieS)&(df.year.between(bornes[0], bornes[1]))].copy()
    dfmoy = dfAjout.groupby(['serie', 'unite', 'zone', 'source'], as_index=False).value.mean()
    dfAjout = dfAjout.merge(dfmoy, 'left', on=['serie','unite','zone','source'], suffixes=['_suppr', None])
    dfAjout['serie'] = serieR
    dfAjout = dfAjout[['serie', 'unite', 'zone', 'year', 'value', 'source']]
    return pd.concat([df, dfAjout], ignore_index=True)

def moy_pond2(df, serieR, lcoeff, lpoids, annee, unite):
    coeff1 = df.loc[df.serie==lcoeff[0]]
    coeff2 = df.loc[df.serie==lcoeff[1]]
    poids1 = df.loc[(df.serie==lpoids[0])&(df.year==annee), ['value', 'zone']]
    poids2 = df.loc[(df.serie==lpoids[1])&(df.year==annee), ['value', 'zone']]
    coeff1 = coeff1.merge(poids1, 'left', on=['zone'], suffixes=[None, '_p1'])
    coeff1['serie'] = serieR
    coeff2 = coeff2.merge(poids2, 'left', on=['zone'], suffixes=['_c2', '_p2'])
    coeff2['serie'] = serieR
    dfAjout = coeff1.merge(coeff2, 'left', on=['serie', 'unite', 'zone', 'year', 'source'])
    dfAjout['value'] = (dfAjout.value * dfAjout.value_p1 + dfAjout.value_c2 * dfAjout.value_p2) / (dfAjout.value_p1 + dfAjout.value_p2)
    dfAjout['unite'] = unite
    dfAjout = dfAjout[['serie', 'unite', 'zone', 'year', 'value', 'source']]
    return pd.concat([df, dfAjout], ignore_index=True)


## Récupération des données Eurostat

Celles-ci sont stockées dans un fichier csv créé à l'aide du notebook __Import Eurostat__. <br> On récupère ce csv dans l'environnement Jupyter afin de définir le dataframe __dfEuro__.

In [163]:
dfEuro = pd.read_csv('donneesEurostat.csv')

On calcule les quatre séries évoquées plus haut ; elles sont ajoutées au dataframe Eurostat.

In [164]:
dfEuro = moy_simple(dfEuro, 'djref', 'dj', bornes=[1990,2019])
dfEuro = moy_simple(dfEuro, 'djrefcli', 'djcli', bornes=[1990,2019])

dfEuro = moy_pond2(dfEuro, 'ipipppxx', ['ipipapxx','ipipapothxx'], ['vadpapxx', 'vadpapothxx'], 2015, 'MEUR2015')
dfEuro = moy_pond2(dfEuro, 'ipiidvxx', ['ipicmpxx','ipicmpothxx'], ['vadcmpxx', 'vadcmpothxx'], 2015, 'MEUR2015')

Enfin on retient uniquement les séries qui nous intéresse, celles qui sont répertoriées dans _l\_series_.

In [165]:
dfEuro = dfEuro[dfEuro.serie.isin(l_series)]
dfEuro

,serie,unite,zone,year,value,source
2046,cpr,MEUR,aut,1990,NaN,Eurostat
2047,cpr,MEUR,aut,1991,NaN,Eurostat
2048,cpr,MEUR,aut,1992,NaN,Eurostat
2049,cpr,MEUR,aut,1993,NaN,Eurostat
2050,cpr,MEUR,aut,1994,NaN,Eurostat
...,...,...,...,...,...,...
115528,ipiidvxx,MEUR2015,ueur28,2016,NaN,Eurostat
115529,ipiidvxx,MEUR2015,ueur28,2017,NaN,Eurostat
115530,ipiidvxx,MEUR2015,ueur28,2018,NaN,Eurostat
115531,ipiidvxx,MEUR2015,ueur28,2019,NaN,Eurostat


## Récupération des données GED

Même principe que pour les données Eurostat : on utilise le csv créé par le notebook __Import GED__ pour obtenir le dataframe __dfGED__ puis on sélectionne les séries voulues à partir de _l\_series_.

In [166]:
dfGED = pd.read_csv('donneesGED.csv')
dfGED = dfGED[dfGED.serie.isin(l_series)]
dfGED

,serie,zone,source,unite,year,value
0,carcfado,aut,NREurostat,Mtoe,1990,0.018745
1,carcfado,aut,NREurostat,Mtoe,1991,0.020827
2,carcfado,aut,NREurostat,Mtoe,1992,0.021869
3,carcfado,aut,NREurostat,Mtoe,1993,0.026034
4,carcfado,aut,NREurostat,Mtoe,1994,0.028117
...,...,...,...,...,...,...
306531,vappd,ueur28,NRD,Mtoe,2017,59.974318
306532,vappd,ueur28,NRD,Mtoe,2018,58.407971
306533,vappd,ueur28,NRD,Mtoe,2019,57.546558
306534,vappd,ueur28,NRD,Mtoe,2020,55.461634


## Fusion et exportation au format xls

On colle les deux dataframes verticalement avec `pd.concat` puis on "pivote" le résultat pour que les années passent en colonnes, conformément au format voulu dans le template.

Enfin on trie les lignes par ordre alphabétique des noms de série et des codes pays avec `sort_values`.

In [167]:
df = pd.concat([dfEuro, dfGED], ignore_index=True)
df = df.pivot(['serie', 'zone', 'source', 'unite'], ['year'], ['value'])
df = df.reset_index()
df.columns = [''.join([s[0], str(s[1])]).replace('value', '') for s in df.columns]
df.sort_values(['serie', 'zone'])
df

,serie,zone,source,unite,1990,1991,1992,1993,1994,1995,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,carcfado,aut,NREurostat,Mtoe,0.018745,0.020827,0.021869,0.026034,0.028117,0.026889,...,0.030278,0.030359,0.029241,0.029145,0.027936,0.027377,0.028826,0.029131,0.029131,NaN
1,carcfado,bel,NREurostat,Mtoe,0.000308,0.000308,0.000308,0.000308,0.000308,0.001541,...,0.003389,0.002978,0.002773,0.002362,0.001849,0.001849,0.001849,0.001849,0.001264,NaN
2,carcfado,bgr,NREurostat,Mtoe,0.038106,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010270,0.011297,0.008216,0.012324,0.019514,0.020121,0.017120,0.011112,0.008331,NaN
3,carcfado,cro,NREurostat,Mtoe,0.002100,0.002100,0.002100,0.005250,0.006300,0.007350,...,0.009450,0.009450,0.009450,0.010500,0.010500,0.010080,0.010185,0.010290,0.005145,NaN
4,carcfado,cyp,NREurostat,Mtoe,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.001938,0.001861,0.001375,0.001271,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12640,vappd,slo,NREurostat,Mtoe,0.196406,0.214129,0.193540,0.197290,0.193683,0.212839,...,0.226884,0.225188,0.195451,0.207441,0.214965,0.225742,0.222498,0.218395,0.219095,NaN
12641,vappd,sui,NRAIE,Mtoe,0.358562,0.409699,0.391093,0.386029,0.389421,0.372391,...,0.480757,0.495853,0.436881,0.484244,0.517970,0.528324,0.520553,0.571128,0.564343,NaN
12642,vappd,swe,NREurostat,Mtoe,1.866231,2.174060,3.537631,3.754507,3.756824,3.895261,...,5.142297,4.985970,4.737924,4.859833,5.108715,5.061303,5.025786,4.958215,4.620195,4.305219
12643,vappd,ueur,NRD,Mtoe,62.359722,61.298297,65.277729,62.716063,55.911728,54.619884,...,59.503169,59.286191,54.953508,55.864274,58.176462,58.381314,56.817306,55.983652,53.898728,53.435336


Dernière étape : on exporte le dataframe sous la forme d'un fichier Excel dont chaque onglet correspond à l'un des pays de _l_pays_.

In [168]:
with pd.ExcelWriter("rawdata-test.xlsx") as writer:
    for pays in l_pays:
        df_pays = df[df.zone == pays]
        df_pays.to_excel(writer, sheet_name=pays, index=False, startrow=3)